In [2]:
import numpy as np
import json
from collections import defaultdict
import networkx as nx
import matplotlib.pyplot as plt

# Greedy QUBO solver

Using the samples of the real devices, the quality of the energy found can be improved by doing bitflips on the bitstrings obtained using a simple greedy algorithm.

In [3]:
def cost_function(x, G):
    obj = 0
    for i, j in G.edges():
        if x[i] * x[j] == 1:
            obj += 2
    return - sum(x) + obj

def greedy_qubo_solver(G, current=None, max_iter=1000):
    nq =  G.number_of_nodes()
    if type(current) == str:
        current = np.array([int(i) for i in current])
    else:
        current = np.random.choice([0, 1], size=nq)
    best = current.copy()
    best_cost = cost_function(best, G)
    bitflips = 0
    for _ in range(max_iter):
        nq_list = list(range(nq))
        # np.random.shuffle(nq_list)
        for i in nq_list:
            candidate = current.copy()
            candidate[i] ^= 1
            current_cost = cost_function(candidate, G)
            if current_cost < best_cost:
                best = candidate.copy()
                best_cost = current_cost
                current = candidate
                bitflips += 1
    best_bitstring = ''.join(str(b) for b in best)
    return best_bitstring, best_cost, bitflips

In [4]:
# backend_name = "quera_aquila"
backend_name = "pasqal_fresnel"

num_sweeps = 2
method = "QAOA"
for case in [1]:
    for nq in [11,13,17,21,25,30,34,41]:
        for extra in [""]:
            with open(f"./Data/{backend_name}/{method}/{case}/{nq}{extra}.json", "r") as file:
                result = json.load(file)
            with open(f"./Data/quera_aquila/{method}/{case}/{nq}{extra}.json", "r") as file:
                result_1 = json.load(file)
            with open(f"./Data/problems/{nq}.json", "r") as file:
                problem = json.load(file)

            G = nx.Graph()
            G.add_nodes_from(range(nq))
            G.add_edges_from(problem["edges"])
            seed = 0
            min_cost = result_1["min_cost"]
            result["greedy_cost"] = defaultdict(int)
            result["greedy"] = defaultdict(int)
            for sample, count in result["samples"].items():
                current_cost = cost_function(np.array([int(i) for i in sample]), G)
                new_sample, new_cost, bitflips = greedy_qubo_solver(G, sample, max_iter=num_sweeps)
                result["greedy_cost"][int(new_cost)] += count
                if new_cost <= min_cost:
                    result["greedy"][bitflips]+= count

            with open(f"./Data/{backend_name}/{method}/{case}/{nq}{extra}.json", "w") as file:
                json.dump(result, file)

# Check if the bitstrings obtained satisfy all the constraints.

In [1]:
def is_valid(sample, G):
    for i, j in G.edges():
        if sample[i] + sample[j] == "11":
            return False
    return True

In [15]:
backend_name = "quera_aquila"
# backend_name = "pasqal_fresnel"

method = "QAOA"
extras = [""]

method = "QAA"
extras = ["_t_2e-06", "_t_4e-06"]

for case in [1,2,3]:
    for nq in [11,13,17,21,25,30,34,41,56,70,84,85]:
        for extra in extras:
            with open(f"./Data/{backend_name}/{method}/{case}/{nq}{extra}.json", "r") as file:
                result = json.load(file)
            with open(f"./Data/quera_aquila/{method}/{case}/{nq}{extra}.json", "r") as file:
                result_1 = json.load(file)
            with open(f"./Data/problems/{nq}.json", "r") as file:
                problem = json.load(file)

            G = nx.Graph()
            G.add_nodes_from(range(nq))
            G.add_edges_from(problem["edges"])
            seed = 0
            min_cost = result_1["min_cost"]
            result["valid"] = 0
            result["cost_valid"] = defaultdict(int)
            for sample, count in result["samples"].items():
                if is_valid(sample, G):
                    result["valid"] += count
                    current_cost = cost_function(np.array([int(i) for i in sample]), G)
                    result["cost_valid"][int(current_cost)] += count


            with open(f"./Data/{backend_name}/{method}/{case}/{nq}{extra}.json", "w") as file:
                json.dump(result, file)